In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# LAB: Evaluación de Modelos

## 1. Introducción

El objetivo de este LAB es realizar la evaluación de un modelos de clasificación partiendo "desde cero". Para ello, usaremos datos de transacciones inmobiliaras en Sacramento.



Importemos los paquetes requeridos:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

plt.style.use('seaborn-white')
%matplotlib inline

Carguemos el dataset de Sacramento:

In [ ]:
sac = pd.read_csv('../Data/Sacramentorealestatetransactions.csv')

Primero hagan un checkeo de la "sanidad" de los datos (por ejemplo, pueden evaluar si hay variables con datos nulos)

In [ ]:
sac.head()

In [ ]:
sac.describe()

In [ ]:
sac.dtypes

In [ ]:
sac.isnull().sum()

In [ ]:
sac.info()

Crear una variable binaria en la cual:

+ $Y = 1$ indica que la casa fue vendida por encima de 200.000 dólares
+ $Y = 0$ indica que la casa fue vendida por igual o menos de 200.000 dólares

Quedarse con las siguientes variables: `number of beds`, `baths`, `sq_ft` y el indicador de precio que generamos en el paso anterior

In [ ]:
cols = ['beds','baths','sq__ft']
X = sac[cols]

In [ ]:
y = sac['price'] > 200000

In [ ]:
y.value_counts()

Dividan el dataset entre training y test sets. Los predictores son `beds`, `baths` y `sq ft`. El target es la variable de precio.
El tamaño del test set debe ser 33% y (opcionalmente) pueden intentar estratificar por el target.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=12)

In [ ]:
# Utilizamos sklearn para estandarizar la matriz de Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

Fitear un modelo de regresión logística:

In [ ]:
logreg = LogisticRegression(C=1e10)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
y_pred = logreg.predict(X_test)

Imprimir la matriz de confusión:

In [ ]:
y_test.value_counts()

In [ ]:
conmat = np.array(confusion_matrix(y_test, y_pred))

confusion = pd.DataFrame(conmat, index=['under_200k', 'over_200k'],
                         columns=['predicted_under_200k','predicted_over_200k'])

print(confusion)

Calcular la proporción de casos correctamente clasificados, la precisión y el recall.

In [ ]:
print('Accuracy=', accuracy_score(y_test, y_pred))
print('Recall=', recall_score(y_test, y_pred))
print('Precision=', precision_score(y_test, y_pred))

¿Qué dicen estas tres medidas acerca del modelo?

Supongamos que un vendedor inmboliario quiere priorizar minimizar los falsos positivos (predecir que una casa se va a vender por encima de \$200.000 cuando en realidad se vende por menos) porque este tipo de errores le hacen "perder plata".

Cambiar el umbral de decisión para **bajar la tasa de falsos positivos** e imprimir nuevamente la matriz de confusión.

In [ ]:
probs = logreg.predict_proba(X_test)
probs_1 = probs[:,1]

In [ ]:
y_pred_2 = probs_1 > 0.7

In [ ]:
confusion_matrix(y_test, y_pred_2)

In [ ]:
print('Accuracy=', accuracy_score(y_test, y_pred_2))
print('Recall=', recall_score(y_test, y_pred_2))
print('Precision=', precision_score(y_test, y_pred_2))

* ¿Cuál sería el lado "negativo" de bajar la tasa de falsos positivos?

Graficar la curva ROC usando las funciones provistas más arriba.

In [ ]:
fpr,tpr,_ = roc_curve(y_test, probs[:,1])
df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
plt.axis([0, 1.01, 0, 1.01])
plt.xlabel('1 - Specificty')
plt.ylabel('TPR / Sensitivity')
plt.title('ROC Curve')
plt.plot(df['fpr'],df['tpr'])
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
plt.show() 

In [ ]:
auc(fpr,tpr)

**Bonus:** ¿Cuándo pueden ser las métricas precision y recall más útiles que la curva ROC?

Tienden a ser más útiles cuando la proporción de casos positivos es menor, dado que ambas métricas son sensibles a esto.

## BONUS: Evaluar otros modelos

Evaluar otros clasificadores además de una regresión logística para este dataset.

In [ ]:
from sklearn.naive_bayes import GaussianNB

nbc = GaussianNB()

In [ ]:
nbc.fit(X_train, y_train)
y_preds_nb = nbc.predict(X_test)
y_probs_nb = nbc.predict_proba(X_test)

In [ ]:
fpr_nb,tpr_nb,thr_nb = roc_curve(y_test, y_probs_nb[:,1])
fpr_log,tpr_log,thr_log = roc_curve(y_test, probs[:,1])

plt.axis([0, 1.01, 0, 1.01])
plt.xlabel('1 - Specificty')
plt.ylabel('TPR / Sensitivity')
plt.title('ROC Curve')
plt.plot(fpr_nb,tpr_nb)
plt.plot(fpr_log,tpr_log)
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
plt.legend(['NB','Logit'])
plt.show()

In [ ]:
print('AUC-NB=', auc(fpr_nb, tpr_nb))
print('AUC-Logit=', auc(fpr_log, tpr_log))